In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import pandas as pd

WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


In [16]:
babies["x"].mean() - babies["y"].mean()

-2.4499999999999993

In [9]:
babies = pd.read_csv("babies.csv")

prior_α = .001
prior_β = .001


with pm.Model() as m:
    α1 = pm.Gamma("μ1", prior_α, prior_β)
    α2 = pm.Gamma("μ2", prior_α, prior_β)
    β1 = pm.Gamma("β1", prior_α, prior_β)
    β2 = pm.Gamma("β2", prior_α, prior_β)
    
    x = pm.Gamma("x", α1, β1, observed=babies["x"])
    y = pm.Gamma("y", α2, β2, observed=babies["y"])
    
    mean_diff = pm.Deterministic("mean_diff", α1/β1 - α2/β2)
    
    trace = pm.sample(
        10000,
        chains=4,
        tune=2000,
        cores=4,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [β2, β1, μ2, μ1]
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 2_000 tune and 10_000 draw iterations (8_000 + 40_000 draws total) took 24 seconds.
There were 165 divergences after tuning. Increase `target_accept` or reparameterize.
There were 179 divergences after tuning. 

In [10]:
az.summary(trace, hdi_prob=.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
μ1,35.089,12.876,12.843,60.315,0.224,0.200,4353.0,2398.0,1.0
μ2,28.049,10.472,9.591,48.444,0.222,0.218,3494.0,1430.0,1.0
β1,3.640,1.344,1.279,6.240,0.023,0.021,4372.0,2425.0,1.0
β2,2.320,0.873,0.761,4.000,0.019,0.018,3516.0,1337.0,1.0
mean_diff,-2.460,0.754,-3.983,-1.028,0.004,0.003,37402.0,28503.0,1.0


In [12]:
wolves = pd.read_csv("wolves.csv")
wolves

,arctic,gender,x3,x7
0,0,0,5.55,2.00
1,0,0,5.94,2.07
2,0,0,5.98,1.94
3,0,0,5.55,1.90
4,0,0,5.63,1.93
5,0,0,5.63,1.83
6,0,1,5.16,1.79
7,0,1,5.12,1.75
8,0,1,4.92,1.63
9,1,0,5.28,1.60


In [18]:
wolves_females_only = wolves.query("gender == 1")
wolves_females_only

,arctic,gender,x3,x7
6,0,1,5.16,1.79
7,0,1,5.12,1.75
8,0,1,4.92,1.63
19,1,1,5.28,1.77
20,1,1,5.24,1.78
21,1,1,5.47,1.85
22,1,1,5.24,1.74
23,1,1,5.75,2.01
24,1,1,5.39,1.77


In [32]:
wolves.iloc[:, 1:].to_numpy().T

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 1.  , 1.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 1.  , 1.  ,
        1.  , 1.  , 1.  ],
       [5.55, 5.94, 5.98, 5.55, 5.63, 5.63, 5.16, 5.12, 4.92, 5.28, 5.87,
        5.59, 5.67, 5.87, 5.63, 5.75, 5.67, 5.55, 5.2 , 5.28, 5.24, 5.47,
        5.24, 5.75, 5.39],
       [2.  , 2.07, 1.94, 1.9 , 1.93, 1.83, 1.79, 1.75, 1.63, 1.6 , 1.86,
        1.77, 1.78, 1.9 , 1.97, 1.88, 1.63, 1.88, 1.65, 1.77, 1.78, 1.85,
        1.74, 2.01, 1.77]])

In [45]:
τ = .01 # precision
μ = 0

with pm.Model() as m:
    data = pm.Data("data", wolves.iloc[:, 1:].to_numpy().T)
    betas = [pm.Normal(f"beta[{i}]", mu=μ, tau=τ) for i in range(4)]
    μ = pm.Deterministic("μ", betas[0] + betas[1] * wolves["gender"] + betas[2] * wolves["x3"] + betas[3] * wolves["x7"])
    y = pm.Logistic("y", μ, observed=data)
                    
    trace2 = pm.sample(
        10000,
        chains=4,
        tune=2000,
        cores=4,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta[3], beta[2], beta[1], beta[0]]
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 2_000 tune and 10_000 draw iterations (8_000 + 40_000 draws total) took 51 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [46]:
az.summary(trace2, hdi_prob=.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],0.379,5.266,-9.796,10.649,0.054,0.038,9468.0,12599.0,1.0
beta[1],0.324,0.608,-0.878,1.504,0.006,0.004,9688.0,11926.0,1.0
beta[2],0.139,1.381,-2.595,2.835,0.016,0.011,7360.0,9228.0,1.0
beta[3],0.546,2.720,-4.719,5.968,0.028,0.020,9581.0,12159.0,1.0
μ[0],2.243,0.596,1.099,3.439,0.006,0.004,9529.0,13531.0,1.0
μ[1],2.336,0.504,1.355,3.327,0.004,0.003,18603.0,21855.0,1.0
μ[2],2.270,0.442,1.417,3.148,0.004,0.003,15567.0,21216.0,1.0
μ[3],2.189,0.389,1.414,2.940,0.004,0.003,11800.0,18988.0,1.0
μ[4],2.216,0.380,1.459,2.950,0.003,0.002,14020.0,21667.0,1.0
μ[5],2.162,0.298,1.590,2.754,0.002,0.001,27100.0,28914.0,1.0


In [47]:
new_data = np.array([[1], [5.28], [1.78]])
new_data

array([[1.  ],
       [5.28],
       [1.78]])

In [48]:
with m:
    pm.set_data({"data": new_data})
    post_pred = pm.sample_posterior_predictive(trace2, var_names=["μ"])

In [49]:
post_pred["μ"]

array([[2.91022786, 2.45416881, 2.19590352, ..., 2.38694777, 2.06569752,
        2.21628415],
       [1.23532782, 1.85097573, 2.08396105, ..., 2.43898403, 3.0231088 ,
        2.67196215],
       [2.95457112, 2.95084389, 2.0861793 , ..., 1.74300785, 2.86250852,
        1.76096007],
       ...,
       [2.11159805, 2.63886368, 2.74800936, ..., 2.53324051, 3.15110295,
        2.73479983],
       [1.96518856, 2.5386851 , 2.71925125, ..., 2.48004718, 3.07162063,
        2.69789203],
       [1.68504322, 1.84937135, 2.27027067, ..., 2.69247844, 2.38173276,
        2.74661887]])

In [50]:
az.summary(post_pred)

/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/arviz/data/base.py:216: UserWarning: More chains (40000) than draws (25). Passed array should have shape (chains, draws, *shape)
  warnings.warn(


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
μ,2.261,0.462,1.39,3.143,0.001,0.001,243639.0,376603.0,1.13


In [2]:
mn = pd.read_csv("micronuclei.csv")
mn.to_numpy()

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [4., 5.],
       [4., 5.],
       [4., 6.]])

In [4]:
from pymc3.math import exp

with pm.Model() as m:
    x_data = pm.Data("x_data", mn["x"].to_numpy())
    y_data = pm.Data("y_data", mn["y"].to_numpy())
    
    beta0 = pm.Normal("beta0", mu=0, tau=.0001)
    beta1 = pm.Normal("beta1", mu=0, tau=.0001)
    
    λ = exp(beta0 + beta1 * x_data)
            
    y = pm.Poisson("y", mu=λ, observed=y_data)
    
    trace3 = pm.sample(
        5000,
        chains=4,
        tune=500,
        cores=4,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )
    
    ppc1 = pm.fast_sample_posterior_predictive(trace3)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta1, beta0]
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 500 tune and 5_000 draw iterations (2_000 + 20_000 draws total) took 15 seconds.
The acceptance probability does not match the target. It is 0.8889928828713357, but should be close to 0.8. Try to increase the n

In [5]:
az.summary(trace3, hdi_prob=.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0,-2.815,0.064,-2.941,-2.688,0.001,0.001,4588.0,4687.0,1.0
beta1,0.671,0.020,0.632,0.709,0.000,0.000,4522.0,4592.0,1.0


In [6]:
az.summary(ppc1, hdi_prob=.95)

/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/arviz/data/base.py:216: UserWarning: More chains (20000) than draws (6000). Passed array should have shape (chains, draws, *shape)
  warnings.warn(


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
y,0.303,0.621,0.0,2.0,0.001,0.001,162229.0,165926.0,1.07


In [17]:
new_data = np.full(6000, 3.5)
with m:
    pm.set_data({"x_data": new_data})
    ppc = pm.sample_posterior_predictive(trace3)
az.summary(ppc, hdi_prob=.95)

/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/arviz/data/base.py:216: UserWarning: More chains (20000) than draws (6000). Passed array should have shape (chains, draws, *shape)
  warnings.warn(


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
y,0.628,0.793,0.0,2.0,0.0,0.0,119856262.0,120017653.0,1.0


https://lucianopaz.github.io/2019/08/19/pymc3-shape-handling/

https://benslack19.github.io/data%20science/statistics/pymc-linreg_entry04/  
better posterior predictive